###### init

In [231]:
# de los resultados generales crea silmulaciones con % de
# datos para que luego sean procesados

In [277]:
# import
from elec_bol20 import *
import elec_bol20.util as ebu
import bokeh.layouts
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import Figure
# output_file, show
import bokeh.tile_providers



In [278]:
df2= ebu.get_dataframe_2020()

In [279]:
df2['COUNT'].sum()

332

In [281]:
df2['HAB']

ID_MESA
110046138042512901    220
110111036112505505    220
110111036112505506    220
110111036112505508    220
110111036112516203    220
                     ... 
112137738202612003    220
112137738202612004    220
112137738202612005    220
112137738202612006     51
112334937922511701    182
Name: HAB, Length: 35600, dtype: int64

In [282]:
bokeh.plotting.output_notebook()
COL_LLEGO = '#aaaaaa'
COL_FALTA = '#db2879'


ll = len(df2)
np.random.seed(100)
df2['xj'] = df2['X'] + np.random.rand(ll) * .5
np.random.seed(200)
df2['yj'] = df2['Y'] + np.random.rand(ll) * .5
cols = ['yj','xj']

s1 = df2[df2['COUNT']][cols]
s2 = df2[~df2['COUNT']][cols]
sr1 = bokeh.models.ColumnDataSource(s1)
sr2 = bokeh.models.ColumnDataSource(s2)


f = bokeh.plotting.figure(output_backend="webgl")
f.scatter(x='xj',y='yj',source = sr2,color=COL_FALTA,radius=.05, alpha=1, legend_label='Mesas faltantes (haz click)')
f.scatter(x='xj',y='yj',source = sr1,color=COL_LLEGO,radius=.05, alpha=1, legend_label='Mesas computadas (haz click)')
f.legend.click_policy="hide"





Loading BokehJS ...

In [283]:
dfn = df2.copy()
dfn.loc[~dfn['BOL'],'DEN_CODES'] = -1

In [284]:
_df = dfn
gr = _df.groupby('DEN_CODES')
den = gr[['HAB']].sum()
den['cum'] = den['HAB'].cumsum()
tot = den['HAB'].sum()
den['mid'] = den['cum'] - den['HAB']/2
den['top'] = 100
den['width'] = den['HAB']-(tot*.01)
den1 = den.copy()

In [285]:
_df = dfn[dfn['COUNT']]
gr = _df.groupby('DEN_CODES')
den = gr[['HAB']].sum()
den['cum'] = den['HAB'].cumsum()
tot = den['HAB'].sum()
den['mid'] = den['cum'] - den['HAB']/2
den['top'] = 1 
den['width'] = den['HAB']-(tot*.01)
den2 = den.copy()

In [286]:
dfn['HAB']

ID_MESA
110046138042512901    220
110111036112505505    220
110111036112505506    220
110111036112505508    220
110111036112516203    220
                     ... 
112137738202612003    220
112137738202612004    220
112137738202612005    220
112137738202612006     51
112334937922511701    182
Name: HAB, Length: 35600, dtype: int64

In [295]:
den1['counted'] = den2['HAB'].astype(np.int64)

In [296]:
den1['top_c'] =den1['counted']/den1['HAB']*100

In [297]:
den1.loc[-1,'mid'] = den1.loc[-1,'mid'] - 1000000

In [357]:
df2[df2['COUNT']]['HAB'].sum()

def _t(s): 
    if np.isnan(s): s=0
    return f'{s:0.1f}'

def _t1(s): 
    if np.isnan(s): s=0
    return f'{s:0.0f}'

def _t2(r):
#     return f'{r["tv"]} ({r["tc"]} %)'
    return f'{r["tc"]}%'

den1['tc']=den1['top_c'].apply(_t)
den1['tv']=den1['counted'].apply(_t1)
den1['text'] = den1.apply(_t2,axis=1)

In [370]:


p = bokeh.plotting.figure(height=400)
# fi = bokeh.plotting.figure()
p.vbar(x=den1['mid'],width=den1['width'],top=den1['top'],color=COL_FALTA)
p.vbar(x=den1['mid'],width=den1['width'],top=den1['top_c'],color=COL_LLEGO)
p.title.text = ''

x = den1[den1.index>=0]['mid'].mean()
p.text(x=[x],y=[120],text=['Nacional'])

x = den1[den1.index==0]['mid'].mean()
p.text(x=[x],y=[101],text=['Densidad baja'],angle=.5, text_font_size="8pt")

x = den1[den1.index==1]['mid'].mean()
p.text(x=[x],y=[101],text=['Densidad media'],angle=.5, text_font_size="8pt")

x = den1[den1.index==2]['mid'].mean()
p.text(x=[x],y=[101],text=['Densidad moderada'],angle=.5, text_font_size="8pt")

x = den1[den1.index==3]['mid'].mean()
p.text(x=[x],y=[101],text=['Densidad alta'],angle=.5, text_font_size="8pt")

x = den1[den1.index<0]['mid'].mean()
p.text(x=[x],y=[110],text=['Exterior'],text_align='center')
p.xaxis.visible = False
p.xgrid.visible = False
p.ygrid.visible = False
p.y_range.start=0
p.y_range.end=130


for l,r in den1.iterrows():
    if l ==-1:
        x = r['mid'] + 400000
    else:
        x = r['mid']
    
    p.text(x=x,y=[r['tc']],text=[r['text']],text_align='center',text_font_size="8pt")

lay = bokeh.layouts.row([p,f])
# lay = p 
bokeh.plotting.show(lay)

In [349]:
den1


,HAB,cum,mid,top,width,counted,top_c,tc,tv,text
DEN_CODES,,,,,,,,,,
-1,301631,301631,-1849184.5,100,228301.75,12678.0,4.203149,4.2,12678,12678 (4.2 %)
0,1222379,1524010,912820.5,100,1149049.75,NaN,NaN,0.0,0,0 (0.0 %)
1,1302649,2826659,2175334.5,100,1229319.75,535.0,0.041070,0.0,535,535 (0.0 %)
2,876558,3703217,3264938.0,100,803228.75,11827.0,1.349255,1.3,11827,11827 (1.3 %)
3,3629708,7332925,5518071.0,100,3556378.75,42846.0,1.180426,1.2,42846,42846 (1.2 %)


In [345]:
den1

,HAB,cum,mid,top,width,counted,top_c,tc,tv,text
DEN_CODES,,,,,,,,,,
-1,301631,301631,-1849184.5,100,228301.75,12678.0,4.203149,4.2,12678,12678 (4.2 %)
0,1222379,1524010,912820.5,100,1149049.75,NaN,NaN,0.0,0,0 (0.0 %)
1,1302649,2826659,2175334.5,100,1229319.75,535.0,0.041070,0.0,535,535 (0.0 %)
2,876558,3703217,3264938.0,100,803228.75,11827.0,1.349255,1.3,11827,11827 (1.3 %)
3,3629708,7332925,5518071.0,100,3556378.75,42846.0,1.180426,1.2,42846,42846 (1.2 %)
